In [1]:
! pip install geopy

In [31]:
import pandas as pd
import numpy as np
from flask import Flask, request, jsonify

In [17]:
df = pd.read_csv('data.csv')

In [18]:
df.head()

,Sector,Label,Address,Activities/Services,Phone Number,Creation Year,City,Website,Entreprise Type,Email
0,Industrie,AErM DISTRIBUTION,"Z.I des Eucalyptus, , ALGER, 16000, Algeria.",Importation et distribution de PDR poids lourd...,+213550416703,2010,alger,http://www.am-company.com,SARL,info@am-company.com
1,Industrie,AD MOTORS,"N°32, Zone industrielle , Alger , 16000, Alge...",équipements de chauffage,+21321526293,2010,alger,admotors-dz.com,SARL,y.hammoutene@admotors-dz.com
2,Industrie,AFRIQUE ENGINS SARL,"Coopérative Camps li N° , Alger, 16000, Algeria.",Importation et distribution de pièces de recha...,+21323707272,2010,alger,NaN,SARL,africengines@yahoo.fr
3,Industrie,ARIBI IMPORT EURL,"Local N°2 Ouled Addi , MÔÇÖsila, 28014, Algeria.",équipements et accessoires pour véhicules • Or...,+21335598327,2010,alger,hotmail.com,SARL,eurlaribi@hotmail.com
4,Industrie,AUTO PARTS EURL,"Cité Douzi 04 n°09, Bab ezouar, Alger, 16000,...",Matériel informatique.,+213550543083,2010,alger,NaN,SARL,hamia.nacer@gmail.com


We Have a problem of using google api to get distance so we make it manually

In [5]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="myuse")
from geopy.distance import geodesic

In [6]:
wilayas=df['City'].unique()
wilayas

array(['alger', 'bejaia', 'tissemsilt', 'el menia', 'constantine',
       'illizi', 'setif', 'biskra', 'tiaret', 'tlemcen', 'mascara',
       'djelfa', 'bordj badji mokhtar', 'el oued', 'el tarf', 'batna',
       'bechar', 'bordj bou arreridj', 'tipaza', 'mila'], dtype=object)

In [7]:
coordinates=[]
for i in wilayas:
    if geolocator.geocode(i)!=None:
        coordinates.append({i:(geolocator.geocode(i).latitude, geolocator.geocode(i).longitude)})
print(coordinates)

[{'alger': (36.7753606, 3.0601882)}, {'bejaia': (36.7511783, 5.0643687)}, {'tissemsilt': (35.785897500000004, 1.8340956752427218)}, {'el menia': (30.7071885, 3.048062480049727)}, {'constantine': (36.3641642, 6.6084281)}, {'illizi': (27.852850500000002, 7.8189636047396895)}, {'setif': (36.1895852, 5.4024656)}, {'biskra': (34.784563500000004, 5.812435334419206)}, {'tiaret': (34.8947575, 1.594579173136212)}, {'tlemcen': (34.881789, -1.316699)}, {'mascara': (35.397838500000006, 0.24301949927219488)}, {'djelfa': (34.342841, 3.217253079090331)}, {'bordj badji mokhtar': (22.626146050000003, 0.12758168507861467)}, {'el oued': (33.215441, 7.155321399098325)}, {'el tarf': (36.67135625, 8.070134002053635)}, {'batna': (35.3384291, 5.731545299000572)}, {'bechar': (31.385726, -2.011595847987405)}, {'bordj bou arreridj': (36.09620285, 4.660274234539607)}, {'tipaza': (36.527157, 2.1672011802712086)}, {'mila': (36.2502135, 6.165216333385166)}]


In [8]:
distances=[]
for i in coordinates:
    b=[]
    for j in coordinates:
        for c in i:
            for k in j:
                b.append({k:geodesic(i[c], j[k]).kilometers})
        sorted_data = sorted(b, key=lambda x: next(iter(x.values())))
    for c in i:
        distances.append({c:sorted_data})
distances[1]

{'bejaia': [{'bejaia': 0.0},
  {'setif': 69.2951234844098},
  {'bordj bou arreridj': 81.21376051575928},
  {'mila': 113.21244302192774},
  {'constantine': 144.74707686046747},
  {'batna': 167.89062178192145},
  {'alger': 178.9638310271025},
  {'biskra': 228.44752640971834},
  {'tipaza': 260.27484796506826},
  {'el tarf': 268.69141310176843},
  {'tissemsilt': 309.3739853825716},
  {'djelfa': 315.342677063474},
  {'tiaret': 375.1095733643548},
  {'el oued': 436.2162276318124},
  {'mascara': 459.463227806316},
  {'tlemcen': 612.6812384203905},
  {'el menia': 695.8879216488067},
  {'bechar': 883.1499614528165},
  {'illizi': 1020.1496322120685},
  {'bordj badji mokhtar': 1636.3336141098052}]}

In [9]:
dists=[]
for i in distances:
    b=[]
    for j in i:
        for k in i[j]:
            for d in k:
                b.append(d)
        dists.append({j:b})
dists[1]

{'bejaia': ['bejaia',
  'setif',
  'bordj bou arreridj',
  'mila',
  'constantine',
  'batna',
  'alger',
  'biskra',
  'tipaza',
  'el tarf',
  'tissemsilt',
  'djelfa',
  'tiaret',
  'el oued',
  'mascara',
  'tlemcen',
  'el menia',
  'bechar',
  'illizi',
  'bordj badji mokhtar']}

We will suggest suppliers to client based on Best Distance and Products 

In [1]:
def AddressesListOfBestSuppliers(selected_wilaya, products):

    #Get top 5 distances for each supplier
    j=0
    k=1
    while(j <len(dists) and k == 1):
        for i in dists[j]:
            if i == selected_wilaya:
                b=dists[j][i][:5] 
                k=0
        j=j+1
    a=[]

    for i in b:
        addresses=df[(df['City'].str.strip() == i) & (df['Activities/Services'].str.strip() == products)]['Address'].values
        a.append(addresses)
    return a


In [30]:
AddressesListOfBestSuppliers('alger','équipements de chauffage')

[array(['N°32, Zone industrielle  , Alger , 16000, Algeria.'], dtype=object),
 array([], dtype=object),
 array([], dtype=object),
 array([], dtype=object),
 array([], dtype=object)]

In [ ]:
@app.route('/GetBestSupliers', methods=['GET'])
def filter_data():
    # Get parameters from the request
    selected_wilaya = request.args.get('wilaya')
    products = request.args.get('product')

    selected_wilaya = 'alger'
    products = 'Association . Organisme.'
    # Call the wilaya2 function to filter the DataFrame
    result_df = wilaya2(selected_wilaya, products)
    print(result_df)
    # Convert the filtered DataFrame to a JSON response
    return jsonify(result_df.to_dict(orient='records'))

if __name__ == '__main__':
    app.run(debug=True)